In [2]:
import sys
from pathlib import Path

APP_DIR = Path().resolve()
if str(APP_DIR) not in sys.path:
    sys.path.insert(0, str(APP_DIR))

from functions import DataFrame, read_csv_parallel, describe, read_csv

# Load dataset
DATA_DIR = APP_DIR.parent / "data"
df = read_csv_parallel(str(DATA_DIR / "us_accidents_clean.csv"), skip_type_inference=False, verbose=True)
print(f"Loaded dataset: {df.shape()}")
df.head(3)


Reading CSV file in parallel using 14 processes...
File size: 0.79 GB, Columns: 18
Processing chunks in parallel...
Parallel reading completed in 14.81s. Combining results...
Combining completed in 0.43s
✓ Total time: 15.24s - Loaded 6,985,065 rows
Loaded dataset: (6985065, 18)


ID  | Severity | Start_Time          | Start_Lat         | Start_Lng          | Distance(mi) | City         | State | Temperature(F) | Visibility(mi) | Wind_Speed(mph) | Precipitation(in) | Weather_Condition | Sunrise_Sunset | Year | Month | DayOfWeek | Hour
----+----------+---------------------+-------------------+--------------------+--------------+--------------+-------+----------------+----------------+-----------------+-------------------+-------------------+----------------+------+-------+-----------+-----
A-2 | 2        | 2016-02-08 06:07:59 | 39.92805900000001 | -82.831184         | 0.01         | Reynoldsburg | OH    | 37.9           | 10.0           | None            | 0.0               | Light Rain        | Night          | 2016 | 2     | Monday    | 6   
A-3 | 2        | 2016-02-08 06:49:27 | 39.063148         | -84.032608         | 0.01         | Williamsburg | OH    | 36.0           | 10.0           | 3.5             | None              | Overcast          | Night        

The following part is just for demonstration of the difference between using parallel (chunking) processes for loading vs normal read_csv function.
read_csv takes a minute more to load compared to read_csv_parallel.

In [3]:
df_test = read_csv(str(DATA_DIR / "us_accidents_clean.csv"), skip_type_inference=False, verbose=True)

Reading CSV file: /Users/dhruvsandu/USC/551-Data Management/Project/data/us_accidents_clean.csv
Columns: 18
  Completed! Loaded 6,985,002 rows.                    


In [4]:
# Basic operations: head, tail, shape, copy
print("Shape:", df.shape())
print("\nFirst 3 rows:")
print(df.head(3))
print("\nLast 3 rows:")
print(df.tail(3))
print("\nCopy test:")
df_copy = df.copy()
print(f"Original: {len(df)} rows, Copy: {len(df_copy)} rows")


Shape: (6985065, 18)

First 3 rows:
ID  | Severity | Start_Time          | Start_Lat         | Start_Lng          | Distance(mi) | City         | State | Temperature(F) | Visibility(mi) | Wind_Speed(mph) | Precipitation(in) | Weather_Condition | Sunrise_Sunset | Year | Month | DayOfWeek | Hour
----+----------+---------------------+-------------------+--------------------+--------------+--------------+-------+----------------+----------------+-----------------+-------------------+-------------------+----------------+------+-------+-----------+-----
A-2 | 2        | 2016-02-08 06:07:59 | 39.92805900000001 | -82.831184         | 0.01         | Reynoldsburg | OH    | 37.9           | 10.0           | None            | 0.0               | Light Rain        | Night          | 2016 | 2     | Monday    | 6   
A-3 | 2        | 2016-02-08 06:49:27 | 39.063148         | -84.032608         | 0.01         | Williamsburg | OH    | 36.0           | 10.0           | 3.5             | None             

In [5]:
# Filter operations
# Filter by value
ca_accidents = df.filter_by_value("State", "CA")
print(f"California accidents: {len(ca_accidents)} rows")

# Filter with condition function
high_severity = df.filter(lambda row: row.get("Severity", 0) >= 4)
print(f"High severity (>=4) accidents: {len(high_severity)} rows")
high_severity.head(3)


California accidents: 1567144 rows
High severity (>=4) accidents: 184709 rows


ID     | Severity | Start_Time          | Start_Lat | Start_Lng   | Distance(mi) | City      | State | Temperature(F) | Visibility(mi) | Wind_Speed(mph) | Precipitation(in) | Weather_Condition | Sunrise_Sunset | Year | Month | DayOfWeek | Hour
-------+----------+---------------------+-----------+-------------+--------------+-----------+-------+----------------+----------------+-----------------+-------------------+-------------------+----------------+------+-------+-----------+-----
A-620  | 4        | 2016-03-11 13:18:48 | 39.917412 | -83.014236  | 0.01         | Columbus  | OH    | 51.8           | 10.0           | 5.8             | None              | Clear             | Day            | 2016 | 3     | Friday    | 13  
A-1198 | 4        | 2016-06-24 22:28:49 | 37.321117 | -121.899887 | 0.0          | San Jose  | CA    | 66.0           | 10.0           | 5.8             | None              | Clear             | Night          | 2016 | 6     | Friday    | 22  
A-1902 | 4        | 2016

In [9]:
print(df.shape())

# Select and drop columns
selected = df.select(["State", "City", "Severity"])
print(f"Selected columns: {selected.columns}")
print(f"Shape: {selected.shape()}")

dropped = df.drop(["Start_Lat", "Start_Lng"])
print(f"\nAfter dropping lat/lng: {dropped.shape()}")



(6985065, 18)
Selected columns: ['State', 'City', 'Severity']
Shape: (6985065, 3)

After dropping lat/lng: (6985065, 16)


In [12]:
# Sort operation
sorted_df = df.sort("Severity", ascending=False)
print("Top 5 accidents by severity:")
sorted_df.head(5).select(["State", "City", "Severity", "Distance(mi)"])


Top 5 accidents by severity:


State | City        | Severity | Distance(mi)
------+-------------+----------+-------------
OH    | Columbus    | 4        | 0.01        
CA    | San Jose    | 4        | 0.0         
CA    | San Bruno   | 4        | 0.0         
CA    | San Jose    | 4        | 0.0         
CA    | San Leandro | 4        | 0.0         

[5 rows x 4 columns]

In [13]:
# Aggregate (single column operations)
print("Total count:", df.aggregate("Severity", "count"))
print("Average severity:", df.aggregate("Severity", "mean"))
print("Max severity:", df.aggregate("Severity", "max"))
print("Min severity:", df.aggregate("Severity", "min"))
print("Average distance:", df.aggregate("Distance(mi)", "mean"))


Total count: 6985065
Average severity: 2.2292474014200296
Max severity: 4
Min severity: 1
Average distance: 0.4608920217626519


In [14]:
# GroupBy with aggregation
state_year = read_csv_parallel(str(DATA_DIR / "us_accidents_state_year.csv"), skip_type_inference=False, verbose=False)

# Group by State and sum accidents
state_totals = state_year.groupby("State").agg({"total_accidents": "sum"})
print("Top 5 states by total accidents:")
state_totals.sort("total_accidents", ascending=False).head(5)


Top 5 states by total accidents:


State | total_accidents
------+----------------
CA    | 1786142        
FL    | 765592         
TX    | 631112         
SC    | 379375         
NY    | 310324         

[5 rows x 2 columns]

In [15]:
# Join operation
df1 = state_year.groupby("State").agg({"total_accidents": "sum"}).head(5)
df2 = state_year.groupby("State").agg({"avg_distance": "mean"}).head(5)

# Join on State
joined = df1.join(df2, on="State", how="left")
print("Joined DataFrames:")
joined


Joined DataFrames:


State | total_accidents | avg_distance       
------+-----------------+--------------------
AL    | 95816           | 0.6785714285714287 
AR    | 18385           | 0.9087999999999998 
AZ    | 153989          | 0.6562068965517242 
CA    | 1786142         | 0.34171428571428575
CO    | 82828           | 0.8844827586206897 

[5 rows x 3 columns]

In [16]:
# Add column
sample = df.head(10)
new_values = [i * 10 for i in range(len(sample))]
df_with_new_col = sample.add_column("multiplier", new_values)
print("DataFrame with new column:")
df_with_new_col.select(["State", "City", "multiplier"])


DataFrame with new column:


State | City         | multiplier
------+--------------+-----------
OH    | Reynoldsburg | 0         
OH    | Williamsburg | 10        
OH    | Dayton       | 20        
OH    | Dayton       | 30        
OH    | Westerville  | 40        
OH    | Dayton       | 50        
OH    | Dayton       | 60        
OH    | Dayton       | 70        
OH    | Westerville  | 80        
OH    | Columbus     | 90        

[10 rows x 3 columns]

In [17]:
# Rename columns
renamed = df.head(5).rename_columns({"State": "state", "City": "city"})
print("Renamed columns:", renamed.columns)
renamed


Renamed columns: ['ID', 'Severity', 'Start_Time', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'city', 'state', 'Temperature(F)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Sunrise_Sunset', 'Year', 'Month', 'DayOfWeek', 'Hour']


ID  | Severity | Start_Time          | Start_Lat         | Start_Lng          | Distance(mi) | city         | state | Temperature(F) | Visibility(mi) | Wind_Speed(mph) | Precipitation(in) | Weather_Condition | Sunrise_Sunset | Year | Month | DayOfWeek | Hour
----+----------+---------------------+-------------------+--------------------+--------------+--------------+-------+----------------+----------------+-----------------+-------------------+-------------------+----------------+------+-------+-----------+-----
A-2 | 2        | 2016-02-08 06:07:59 | 39.92805900000001 | -82.831184         | 0.01         | Reynoldsburg | OH    | 37.9           | 10.0           | None            | 0.0               | Light Rain        | Night          | 2016 | 2     | Monday    | 6   
A-3 | 2        | 2016-02-08 06:49:27 | 39.063148         | -84.032608         | 0.01         | Williamsburg | OH    | 36.0           | 10.0           | 3.5             | None              | Overcast          | Night        

In [18]:
# Fillna operation
sample_df = df.head(10).select(["State", "City", "Severity"])
# Simulating missing values
sample_df.data["Severity"][0] = None
sample_df.data["Severity"][1] = None

filled = sample_df.fillna_column("Severity", 0)
print("Before fillna:", sample_df.data["Severity"][:3])
print("After fillna:", filled.data["Severity"][:3])


Before fillna: [None, None, 3]
After fillna: [0, 0, 3]


In [19]:
# Convert column type
sample = df.head(10).select(["Severity", "Distance(mi)"])
print("Original types:")
print(f"Severity sample: {sample.data['Severity'][:3]}")
print(f"Distance sample: {sample.data['Distance(mi)'][:3]}")

# Convert to string
converted = sample.convert_column_type("Severity", str)
print("\nAfter converting Severity to string:")
print(converted.data["Severity"][:3])


Original types:
Severity sample: [2, 2, 3]
Distance sample: [0.01, 0.01, 0.01]

After converting Severity to string:
['2', '2', '3']


In [22]:
# Round column
sample = state_year.head(10).select(["State", "avg_distance"])
print("Before rounding:")
print(sample.data["avg_distance"][:3])

rounded = sample.round_column("avg_distance", decimals=1)
print("\nAfter rounding to 1 decimal:")
print(rounded.data["avg_distance"][:3])


Before rounding:
[0.68, 2.5, 0.0]

After rounding to 1 decimal:
[0.7, 2.5, 0.0]


In [24]:
# GroupBy convenience methods (count, sum, mean, min, max)
grouped = state_year.groupby("State")

# Count
count_result = grouped.count()
print("Count per state (first 5):")
print(count_result.head(5))

# Sum
sum_result = grouped.sum("total_accidents")
print("\nSum of accidents per state (first 5):")
print(sum_result.head(5))


Count per state (first 5):
State | count
------+------
AL    | 28   
AR    | 25   
AZ    | 29   
CA    | 35   
CO    | 29   

[5 rows x 2 columns]

Sum of accidents per state (first 5):
State | total_accidents
------+----------------
AL    | 95816          
AR    | 18385          
AZ    | 153989         
CA    | 1786142        
CO    | 82828          

[5 rows x 2 columns]
